# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import GEOparse as Geo
import seaborn as sns
from pandas.core.ops import comp_method_OBJECT_ARRAY
from collections import Counter

# from sanbomics import annotate
os.getcwd()

'/home/max/mcrc-cetuximab-analysis/notebooks'

In [33]:
fpkms_df = pd.read_csv("../raw/GSE183984_ASAN_RNASEQ_FPKM_ensg.csv", index_col=0).T
raw_counts_df = pd.read_csv("../raw/GSE183984_ASAN_RNASEQ_raw_counts_ensg.csv", index_col=0).T

print(fpkms_df.shape)
print(raw_counts_df.shape)

# columns are the same in TPM and FPKM
print(set(fpkms_df.columns) - set(raw_counts_df.columns))
print(set(raw_counts_df.columns) - set(fpkms_df.columns))

# no duplicate column names
print(len(fpkms_df.columns), len(set(fpkms_df.columns)))
print(len(raw_counts_df.columns), len(set(raw_counts_df.columns)))

# no transcript level distinction
print(''.join(list(raw_counts_df.columns)).find('T')) 

data_ensemble_genes = list(fpkms_df.columns)

cnt = Counter()
for gene in data_ensemble_genes:
    cnt[gene[3]] += 1
cnt

(113, 58735)
(113, 58735)
set()
set()
58735 58735
58735 58735
-1


Counter({'G': 58735})

In [45]:
def parse_gtf(gtf_path):
    ensemble_gene_id_to_length = Counter()
    ensemble_gene_id_to_version = dict()
    ensemble_gene_id_to_hgnc = dict()
    with open(gtf_path, "r") as f:
        for li, line in enumerate(f):
            # print(line)
            if line.startswith("#"):
                continue
            
            parts = line.strip().split()
            chromosome = parts[0]
            entry_source_0 = parts[1]
            entry_type = parts[2]
            start = int(parts[3])
            end = int(parts[4])
            info = dict()
            for i in range(8, len(parts) - 1, 2):
                key = parts[i]
                val = parts[i + 1][1:-2]
                if key == 'gene_name' and val == 'havana':
                    print(key, val, i)
                info[key] = val
            
            if entry_type == 'gene' and 'gene_id' in info and 'gene_name' in info:
                ensemble_gene_id_to_hgnc[info['gene_id']] = info['gene_name']
            elif entry_type == 'exon':
                version = info['gene_version']
                ensemble_gene_id = info['gene_id']
                ensemble_gene_id_to_version[ensemble_gene_id] = version
                ensemble_gene_id_to_length[ensemble_gene_id] += int(parts[4]) - int(parts[3]) + 1
                    
    return ensemble_gene_id_to_hgnc, ensemble_gene_id_to_length, ensemble_gene_id_to_version

ensemble_gene_id_to_hgnc, ensemble_gene_id_to_length, ensemble_gene_id_to_version = parse_gtf('../data/Homo_sapiens.GRCh38.113.gtf')

In [46]:
print(len(ensemble_gene_id_to_hgnc.keys()))
print(len(data_ensembl_genes))
print(len(set(ensemble_gene_id_to_hgnc.keys()) - set(data_ensembl_genes)))
print(len(set(data_ensembl_genes) - set(ensemble_gene_id_to_hgnc.keys())))

ensemble_genes_common = list(set(data_ensemble_genes).intersection(ensemble_gene_id_to_hgnc.keys()))
len(ensemble_genes_common)

42745
58735
597
16587


42148

In [48]:
from collections import Counter

cnt = Counter()
hgnc_to_ensemble_set = dict()
for key in ensemble_genes_common:
    hgnc = ensemble_gene_id_to_hgnc[key]
    cnt[hgnc] += 1
    if cnt[hgnc] == 1:
        hgnc_to_ensemble_set[hgnc] = {key}
    else:
        hgnc_to_ensemble_set[hgnc].add(key)


for key in hgnc_to_ensemble_set:
    if len(hgnc_to_ensemble_set[key]) > 1 and key is not None:
        print(key)
        print([(egene, ensemble_lengths[egene]) for egene in hgnc_to_ensemble_set[key]])

Metazoa_SRP
[('ENSG00000273710', 286), ('ENSG00000276735', 314), ('ENSG00000273725', 285), ('ENSG00000278770', 281), ('ENSG00000278794', 267), ('ENSG00000280502', 297), ('ENSG00000275213', 290), ('ENSG00000277396', 262), ('ENSG00000278100', 322), ('ENSG00000275460', 260), ('ENSG00000278764', 282), ('ENSG00000274589', 305), ('ENSG00000277721', 254), ('ENSG00000274799', 282), ('ENSG00000275901', 254), ('ENSG00000276359', 284), ('ENSG00000275362', 320), ('ENSG00000278724', 281), ('ENSG00000283389', 244), ('ENSG00000278109', 288), ('ENSG00000277740', 289), ('ENSG00000278250', 362), ('ENSG00000277051', 311), ('ENSG00000276839', 269), ('ENSG00000277617', 260), ('ENSG00000276532', 303), ('ENSG00000258045', 268), ('ENSG00000277578', 277), ('ENSG00000277473', 277), ('ENSG00000274459', 279), ('ENSG00000274701', 298), ('ENSG00000274079', 283), ('ENSG00000275041', 284), ('ENSG00000277771', 265), ('ENSG00000283636', 248), ('ENSG00000276823', 327), ('ENSG00000275319', 316), ('ENSG00000278226', 260),

In [53]:
parsed_hgncs = set(hgnc_to_ensemble_set.keys())
bg_hgncs = set(open('../data/gnames.txt').read().strip().split())

print('Excess genes: ', len(parsed_hgncs - bg_hgncs))
print('Missing genes from BG list: ', len(bg_hgncs - parsed_hgncs))

common_genes = parsed_hgncs.intersection(bg_hgncs)
print('Common genes: ', len(common_genes))

Excess genes:  22540
Missing genes from BG list:  1657
Common genes:  18405


# Examine raw counts/FPKM data

# Rename columns and shrink gene list to those that are in BG list

In [ ]:
from time import time
import mygene

mg = mygene.MyGeneInfo()

# Example list of Ensembl gene IDs
ensembl_ids = raw_counts_df.columns

t1 = time()
# Query
result = mg.querymany(ensembl_ids, scopes="ensembl.gene", fields="symbol", species="human")
t2 = time()
print(t2-t1)


# Convert to dict: {ensembl_id: gene_symbol}
ensembl_to_hgnc = {item['query']: item.get('symbol', None) for item in result}

In [ ]:

print('Total genes: ', len(ensembl_to_hgnc))
print('Genes could convert: ', sum(ensebml_to_hgnc[k] != None for k in ensebml_to_hgnc))

# renaming to new names
raw_counts_df = raw_counts_df.rename(columns=ensebml_to_hgnc)
fpkms_df = fpkms_df.rename(columns=ensebml_to_hgnc)

# dropping cols that couldnt convert
raw_counts_df = raw_counts_df.drop(columns=[None])
fpkms_df = fpkms_df.drop(columns=[None])

print('With/without dup cols: ', len(fpkms_df.columns), len(set(fpkms_df.columns)))

# deduplicating cols
raw_counts_df = raw_counts_df.loc[:, ~raw_counts_df.columns.duplicated()]
fpkms_df = fpkms_df.loc[:, ~fpkms_df.columns.duplicated()]

In [ ]:
with open('../data/gnames.txt', 'r') as f:
    bg_genes = f.read().strip().split()
    print('BG list contains genes: ', len(bg_genes))

common_with_bg = set(raw_counts_df.columns).intersection(set(bg_genes))
print('Common genes in renamed data and BG list: ', len(common_with_bg))
gene_list = common_with_bg

In [ ]:
# using only those in BG list
raw_counts_df = raw_counts_df[list(common_with_bg)]
fpkms_df = fpkms_df[list(common_with_bg)]
raw_counts_df.shape

# Get gene lengths

In [ ]:
def counts_to_tpm(counts):
    gl_df = pd.Series(hgnc_gene_lengths)
    tpm = counts / (gl_df / 1000)
    tpm = (tpm.T / tpm.sum(axis=1)).T * 1_000_000
    print('Nans: ', tpm.isna().sum().sum())
    # print(tpm)
    return tpm

def counts_to_fpkm(counts):
    gl_df = pd.Series(hgnc_gene_lengths)
    fpkm = (counts.T / counts.sum(axis=1)).T * 1_000_000
    fpkm = fpkm / (gl_df / 1000)
    print('Nans: ', fpkm.isna().sum().sum())
    # print(tpm)
    return fpkm

def fpkm_to_tpm(fpkm):
    return (fpkm.T / fpkm.sum(axis=1)).T * 1_000_000

print(raw_counts_df.shape)

tpm_df = counts_to_tpm(raw_counts_df)
tpm_from_fpkm_df = fpkm_to_tpm(raw_counts_df)

fpkm_from_counts_df = counts_to_fpkm(raw_counts_df)


In [ ]:
fpkm_from_counts_df

In [ ]:
fpkms_df

In [ ]:
tpm_df

In [ ]:
tpm_from_fpkm_df

In [ ]:
# should be sum 1 in every row
print((fpkm_from_counts_df * (pd.Series(hgnc_gene_lengths) / 1000)).sum(axis=1) / 10 ** 6)
print((fpkms_df * (pd.Series(hgnc_gene_lengths) / 1000)).sum(axis=1) / 10 ** 6)

In [ ]:
log_tpm_df = tpm_df.apply(lambda x: np.log(1 + x))
plt.figure(figsize=(16, 9))
sns.histplot(log_tpm_df.iloc[1])
plt.ylim(0, 1000)

In [ ]:
plt.figure(figsize=(16, 9))
sns.histplot(log_tpm_df.mean())
plt.ylim(0, 1000)